### <mark>**✅Crawling_Instagram**

##### **Library**

In [60]:
# selenium import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# library import
from bs4 import BeautifulSoup
from dateutil import parser
import time
import pandas as pd
import platform
import os
from time import sleep
from glob import glob
from tqdm import tqdm
from datetime import datetime
import pickle

##### **Accept**

In [61]:
os_name = platform.system().lower()
architecture = platform.machine()

driver_path = None
if os_name == 'darwin':
    driver_path = glob('../driver/**/chromedriver', recursive=True)[0]
else:
    driver_path = glob('../driver/**/chromedriver.exe', recursive=True)[0]

In [62]:
# 드라이버 파일의 권한을 확인하고 수정
if os_name == 'darwin':  # 맥 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가
else:  # 윈도우 사용자
    driver_permissions = os.stat(driver_path).st_mode
    if not (driver_permissions & 0o100):  # 실행 권한 확인
        os.chmod(driver_path, driver_permissions | 0o111)  # 실행 권한 추가

##### **Chrome Options**

In [63]:
service = Service(executable_path=driver_path)
chrome_options =  webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

##### **Define**

In [65]:
url = 'https://www.instagram.com/p/C052WSVpeg7/'

In [67]:
# Web OPEN
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)

driver.refresh()
time.sleep(3)
input()

''

In [69]:
try:
    comment_container = driver.find_element(By.XPATH, '//*[@id="mount_0_0_vu"]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]')
    
    last_height = driver.execute_script("return arguments[0].scrollHeight", comment_container)
    
    while True:
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", comment_container)
        time.sleep(2)

        new_height = driver.execute_script("return arguments[0].scrollHeight", comment_container)

        if new_height == last_height:
            break
        last_height = new_height

except Exception as e:
    print("❌ 댓글 컨테이너 접근 실패:", e)

❌ 댓글 컨테이너 접근 실패: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.166)
Stacktrace:
	GetHandleVerifier [0x00007FF7BF6FFE45+26629]
	(No symbol) [0x00007FF7BF666010]
	(No symbol) [0x00007FF7BF4F931A]
	(No symbol) [0x00007FF7BF4E4F65]
	(No symbol) [0x00007FF7BF509E04]
	(No symbol) [0x00007FF7BF57F85F]
	(No symbol) [0x00007FF7BF59FA55]
	(No symbol) [0x00007FF7BF577883]
	(No symbol) [0x00007FF7BF540550]
	(No symbol) [0x00007FF7BF541803]
	GetHandleVerifier [0x00007FF7BFA572BD+3529853]
	GetHandleVerifier [0x00007FF7BFA6DA22+3621858]
	GetHandleVerifier [0x00007FF7BFA624D3+3575443]
	GetHandleVerifier [0x00007FF7BF7CB77A+860474]
	(No symbol) [0x00007FF7BF67088F]
	(No symbol) [0x00007FF7BF66CBC4]
	(No symbol) [0x00007FF7BF66CD66]
	(No symbol) [0x00007FF7BF65C2C9]
	BaseThreadInitThunk [0x00007FFA170FE8D7+23]
	RtlUserThreadStart [0x00007FFA1743BF6C+44]



In [41]:
comment_blocks = driver.find_elements(By.XPATH,
    '//*[@id="mount_0_0_uo"]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[2]/div'
)

In [44]:
data = []

for idx, block in enumerate(comment_blocks, 1):
    try:
        user = block.find_element(By.XPATH, './/span[1]//a//span').text
    except:
        user = None

    try:
        date = block.find_element(By.XPATH, './/span[2]//a//time').text
    except:
        date = None

    try:
        text = block.find_element(By.XPATH, './/div[contains(@class, "x1cy8zhl")]/span').text
    except:
        text = None

    data.append({
        "ID": user,
        "CREATED": date,
        "COMMENT": text
    })

##### **To CSV**

In [56]:
result_df = pd.DataFrame(data)

In [58]:
driver.close()
driver.quit()

In [19]:
today_str = datetime.today().strftime("%m%d")

In [23]:
result_df.to_csv(rf'..\data\review\{today_str}_{file_name}.csv', index=False, encoding='utf-8-sig')